In [ ]:
# https://towardsdatascience.com/drowsiness-detection-using-convolutional-neural-networks-face-recognition-and-tensorflow-56cdfc8315ad
%pip install sklearn
%pip install -U -q segmentation-models
%pip install tensorflow==2.1.0
%pip install -q keras==2.3.1
%pip install -q tensorflow-estimator==2.1.

## Imports libs
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ["SM_FRAMEWORK"] = "tf.keras"

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense

import segmentation_models as sm
import cv2
import os
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
#from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
#from keras.models import Sequential
#from keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,Dropout

#import keras

from tensorflow import keras

from tensorflow.keras.callbacks import ModelCheckpoint,CSVLogger
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras import layers


def loadImages(folder, eyes = 0):
    count = 0
    error_count = 0
    images = []
    for filename in os.listdir(folder):
        try:
            image = cv2.imread(os.path.join(folder, filename))
            image = cv2.resize(image, (224,224)) 

            images.append([image, eyes])
        except:
            error_count += 1
            print('Error count = ' + str(error_count))
            continue
        
        count += 1
        if count % 500 == 0:
            print('Import Image Successful Count = ' + str(count))

    return images

folder="data/train/open"
open_eyes = loadImages(folder, 0)

folder="data/train/close" 
closed_eyes = loadImages(folder, 1)   
eyes = closed_eyes + open_eyes

X = [] 
y = [] 
for features, label in eyes: 
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, 224, 224, 3)

y = np.array(y)

X = X / 255.0

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y)

model = tf.keras.applications.vgg16.VGG16()

model.summary()

for layer in model.layers:
    layer.trainable = False

base_input = model.layers[0].input 

base_output = model.layers[-5].output

Flat_layer = layers.Flatten()(base_output)
final_output = layers.Dense(1)(Flat_layer) ## one node (1 or 0) == Open or closed
fina_output = layers.Activation('sigmoid')(final_output) # Binary classification thus using sigmoid

model = keras.Model(inputs = base_input, outputs = final_output)

model.summary()

model.compile(loss='binary_crossentropy', 
                  optimizer='adam', 
                  metrics=['acc'])

# Create an Instance of Early Stopping Callback
#early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 10, mode = 'min', restore_best_weights = True)


# fitting the model
history = model.fit(X_train,
            y_train,
            batch_size=800,
            validation_data=(X_test, y_test),
            epochs=40)

#history = new_model.fit(x = features_train, y = labels_train, epochs = 40, batch_size = 4, shuffle = True, validation_split = 0.2, callbacks = [early_stopping_callback])

# evaluate the model 
model.evaluate(X_test, y_test, verbose=1)

model.save('cnn_model_transfer_learning_VGG16.h5')

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Segmentation Models: using `tf.keras` framework.
Import Image Successful Count = 500
Import Image Successful Count = 1000
Import Image Successful Count = 500
Import Image Successful Count = 1000
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928    

In [ ]:
from sklearn import metrics 
from sklearn.metrics import classification_report, confusion_matrix 


predictions = model.predict(X_test)  

In [ ]:
#print(predictions)

In [ ]:
predictions = np.where(predictions > 0.5, 1, 0)
#print(predictions)


In [ ]:
# Confusion Matrix
print("Accuracy", metrics.accuracy_score(y_test, predictions))
print(confusion_matrix(y_test,predictions))  
print(classification_report(y_test,predictions))  

In [ ]:
# evaluate the model 
model.evaluate(X_test, y_test, verbose=1)

In [ ]:
# Model Summary
print(model.summary())


In [ ]:
#conda install pydot pydotplus
#from keras.utils import plot_model
%pip install pydot 
%pip install pydotplus
%pip install graphviz
from tensorflow.keras.utils import plot_model

#from keras.utils.vis_utils import plot_model
import IPython
import keras

plot_model(model, to_file='model.png',show_shapes=True)
IPython.display.Image("model.png")

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

In [ ]:
for layer in model.layers:
    weights = layer.get_weights()
    print(weights)

In [ ]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())